# Introduccion a RDDs Clave-Valor

- Muchos conuuntos de datos que vemos en ejemplos de la vida real suelen ser pares clave-valor
- Ejemplos:
Un conjunto de datos que contiene numeros de pasaporte y los nombres de los propietarios.
- El patron tipico de este tipo de conjunto de datos es que cada fila es una clave que esta asociada a uno o multiples valores
- Un RDD clave-valor es un tipo particular de RDD que puede almacenar pares Clave-Valor
- Son bloques utiles de construccion en muchos programas Spark

### Como crear RDDs clave-valor

1. Crear RDDs clave-valor a partir de una lista de datos clave-valor estructurada llamada tupla

In [ ]:
// Creamos una tupla
val tuple = ("Juan", 25)

val name = tuple._1
val age = tuple._2

### Convertir tuplas en RDDs clave-valor

In [ ]:
// Ejemplo 1

val tuple = List(("Lily", 23), ("Jack", 29), ("Mary", 29), ("James", 8))
val pairRDD = sc.parallelize(tuple)

pairRDD.coalesce(1).saveAsTextFile("out/pair_rdd_from_tuple_list")

In [ ]:
// Ejemplo 2

val inputStrings = List("Lily 23", "Jack 29", "Mary 29", "James 8")
val regularRDDs = sc.parallelize(inputStrings)

val pairRDD = regularRDDs.map(s => (s.split(" ")(0), s.split(" ")(1)))
pairRDD.coalesce(1).saveAsTextFile("out/pair_rdd_from_regular_rdd")


# Transformaciones Filter y MapValue en RDDs clave-valor

- Los RDDs clave-valor pueden utilizar todas las transformaciones disponibles para RDDs regulares, por lo tanto admiten las mismas funciones
- Dado que los RDDs clave-valor tienen tuplas, tenemos que ejecutar las funciones que operan en tuplas en lugar de elementos individuales


### Transformacion Filter

- La transformacion Filter tambien se puede aplicar a un RDD clave-valor
- La transformacion Filter se ejecutaa como una funcion y genera un RDD clave-valor formado por aquellos elementos seleccionados que pasan la funcion filter

#### Ejercicio:

Cree un programa Spark para leer los datos del aeropuerto desde in / airports.text; generar un par RDD con el nombre del aeropuerto como clave y el nombre del país como valor. Luego, elimine todos los aeropuertos que se encuentran en Estados Unidos y envíe el par RDD a out / airports_not_in_usa_pair_rdd.text Cada fila del archivo de entrada contiene las siguientes columnas:  Identificación del aeropuerto, nombre del aeropuerto, ciudad principal a la que sirve el aeropuerto, país donde se encuentra el aeropuerto, Código IATA / FAA, Código ICAO, Latitud, Longitud, Altitud, Zona horaria, DST, Zona horaria en formato Olson

#### Solucion:

In [ ]:
val airportsRDD = sc.textFile("input/airports.text")

val airportPairRDD = airportsRDD.map(line => (line.split(",")(1),
      line.split(",")(3)))
val airportsNotInUSA = airportPairRDD.filter(keyValue => keyValue._2 != "\"United States\"")

airportsNotInUSA.saveAsTextFile("out/airports_not_in_usa_pair_rdd.text")

# Transformaciones map y mapValues

- Transformaciones map tambien funciona para RDDs clave-valor. Se puede utilizar para convertir un RDD en otro.
- Sin embargo, cuando se trabaja con RDDs clave-valor, solo queremos acceder al valor de nuestro par clave-valor

#### Ejercicio:


Cree un programa Spark para leer los datos del aeropuerto desde / airports.text, genere un par RDD con el nombre del aeropuerto siendo la clave y el nombre del país el valor. Luego convierta el nombre del país a mayúsculas y salida del par RDD a out / airports_uppercase.text Cada fila del archivo de entrada contiene las siguientes columnas: Identificación del aeropuerto, nombre del aeropuerto, ciudad principal a la que sirve el aeropuerto, país donde se encuentra el aeropuerto, código IATA / FAA, Código OACI, latitud, longitud, altitud, zona horaria, horario de verano, zona horaria en formato Olson

#### Solucion:

In [ ]:
val airportsRDD = sc.textFile("input/airports.text")

val airportPairRDD = airportsRDD.map((line: String) => (line.split(",")(1),
      line.split(",")(3)))

val upperCase = airportPairRDD.mapValues(countryName => countryName.toUpperCase)

upperCase.saveAsTextFile("out/airports_uppercase.text")

# Agregaciones reduceByKey

- Cuando nuestro conjunto de datos se describe en el formato de pares clave-valor, es bastante comun querer agregar estadisticas a traves de todos los elementos con la misma clave
- reduceByKey ejecuta varias operaciones reduce paralelas, una para cada clave en el conjunto de datos, donde cada operacion combina valores que tienen la misma clave
- Teniendo en cuenta que los conjuntos de entrada podrian tener un gran numero de claves, la operacion reduceByKey no esta implementada como una accion que genera un valor al programa driver. En su lugar, genera un nuevo RDD que consta de clave y el valor reducido para esa clave

# GroupByKey





- Un uso comun en el caso de RDDs clave-valor es la agrupacion de datos por claves por ejemplo: Ver juntas todas las transacciones de una cuenta
- Si nuestros datos ya fueron introducidos de la forma deseada,  groupByKey agrupara nuestros datos usando la clave de nuestro RDD clave-valor
- Digamos que el RDD clave-valor de input tiene claves del tipo K y valores de tipo V, si ejecutamos groupByKey en este RDD, obtendremos un nuevo RDD clave-valor de tipo K e iterable V

Ejercicio:


- Cree un programa Spark para leer los datos del aeropuerto desde / airports.text, muestra la lista de los nombres de los aeropuertos ubicados en cada país. Cada fila del archivo de entrada contiene las siguientes columnas: Identificación del aeropuerto, nombre del aeropuerto, ciudad principal a la que sirve el aeropuerto, país donde se encuentra el aeropuerto, código IATA / FAA, Código OACI, latitud, longitud, altitud, zona horaria, horario de verano, zona horaria en formato Olson

#### Solucion:

In [ ]:
val lines = sc.textFile("input/airports.text")

val countryAndAirportNameAndPair = lines.map(airport => (airport.split(",")(3),
                                                             airport.split(",")(1)))
val airportsByCountry = countryAndAirportNameAndPair.groupByKey()

for ((country, airportName) <- airportsByCountry.collectAsMap()) println(country + ": " + airportName.toList)
  

#### Ejemplo 2

GroupByKey Vs ReduceByKey





In [ ]:
val words = List("one", "two", "two", "three", "three", "three")
val wordsPairRdd = sc.parallelize(words).map(word => (word, 1))

val wordCountsWithReduceByKey = wordsPairRdd.reduceByKey((x, y) => x + y).collect()
println("wordCountsWithReduceByKey: " + wordCountsWithReduceByKey.toList)

val wordCountsWithGroupByKey = wordsPairRdd.groupByKey().mapValues(intIterable => intIterable.size).collect()
println("wordCountsWithGroupByKey: " + wordCountsWithGroupByKey.toList)

###### ReduceByKey funciona mucho mejor que GroupByKey

# Transformacion sortByKey

- Podemos ordenar un RDD clave-valor siempre y cuando halla un orden definido en la clave
- Una vez que hayamos ordenado nuestro RDD clave-valor, cualquier llamado sobre el RDD clave-valor ordenado, ya sea copiar o guardar, nos retornara datos ordenados

In [4]:
val rdd = sc.parallelize(Seq(
                            ("math",    55),
                            ("math",    56),
                            ("english", 57),
                            ("english", 58),
                            ("science", 59),
                            ("science", 54)))
 rdd.collect()
val sorted1 = rdd.sortByKey()
sorted1.collect()

rdd: org.apache.spark.rdd.RDD[(String, Int)] = ParallelCollectionRDD[8] at parallelize at <console>:25
sorted1: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[11] at sortByKey at <console>:33
res2: Array[(String, Int)] = Array((english,57), (english,58), (math,55), (math,56), (science,59), (science,54))


In [3]:
val rdd1 = sc.parallelize(Seq(
    ("India",91),
    ("USA",1),
    ("Brazil",55),
    ("Greece",30),
    ("China",86),
    ("Sweden",46),
    ("Turkey",90),
    ("Nepal",977)))
val rdd2 = rdd1.sortByKey()
rdd2.collect()

rdd1: org.apache.spark.rdd.RDD[(String, Int)] = ParallelCollectionRDD[4] at parallelize at <console>:28
rdd2: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[7] at sortByKey at <console>:37
res1: Array[(String, Int)] = Array((Brazil,55), (China,86), (Greece,30), (India,91), (Nepal,977), (Sweden,46), (Turkey,90), (USA,1))


# Operaciones Join en Spark

- Las operaciones Join nos permiten unir dos RDDs que son probalemente una de las operaciones mas comunes con RDDs clave-valor

![title](images\operation-join.png)



#### Ejemplos:

In [5]:
val ages = sc.parallelize(List(("Tom", 29),("John", 22)))
val addresses = sc.parallelize(List(("James", "USA"), ("John", "UK")))

val join = ages.join(addresses)
join.saveAsTextFile("out/age_address_join.text")

val leftOuterJoin = ages.leftOuterJoin(addresses)
leftOuterJoin.saveAsTextFile("out/age_address_left_out_join.text")

val rightOuterJoin = ages.rightOuterJoin(addresses)
rightOuterJoin.saveAsTextFile("out/age_address_right_out_join.text")

val fullOuterJoin = ages.fullOuterJoin(addresses)
fullOuterJoin.saveAsTextFile("out/age_address_full_out_join.text")

ages: org.apache.spark.rdd.RDD[(String, Int)] = ParallelCollectionRDD[12] at parallelize at <console>:25
addresses: org.apache.spark.rdd.RDD[(String, String)] = ParallelCollectionRDD[13] at parallelize at <console>:26
join: org.apache.spark.rdd.RDD[(String, (Int, String))] = MapPartitionsRDD[16] at join at <console>:28
leftOuterJoin: org.apache.spark.rdd.RDD[(String, (Int, Option[String]))] = MapPartitionsRDD[20] at leftOuterJoin at <console>:31
rightOuterJoin: org.apache.spark.rdd.RDD[(String, (Option[Int], String))] = MapPartitionsRDD[24] at rightOuterJoin at <console>:34
fullOuterJoin: org.apache.spark.rdd.RDD[(String, (Option[Int], Option[String]))] = MapPartitionsRDD[28] at fullOuterJoin at <console>:37


#### Mejores Practicas

- Si ambos RDDs tienen claves duplicadas, la operacion Union puede ampliar dramaticamente el tamaño de los datos. Se recomienda ejecutar una operacion *distinct* o *combineByKey* para disminuir el tamaño de las claves si fuera posible
- La operacion *Join* puede requerir grandes transferencias de datos o incluso crear conjuntos de datos fuera de nuestras capacidades para administrarlos
- Las uniones en general son muy costosas y requieren que las claves de los RDDs esten situadas en la misma particion del disco para que pueda combinarse